## Load data

In [15]:
# Import the dataset loading function from sklearn
from sklearn.datasets import fetch_openml

# Load the titanic dataset from openml
titanic = fetch_openml(name='titanic', version=1, as_frame=True)

In [16]:
data = titanic.data
data.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"


## Feature engineering

### Name

### Family size

### Deck

### Missing data imputation

#### Sensible value imputation

#### Predictive imputation

### Age-dependent variables

#### Child

#### Mother

## Build up a Random Forest & a GBDT classifier

### random forest classfier 

### gradient boosted decision tree classifier

## Fine-Tuning: tune the tree depth hyperparameter in the SVM classifier

## Retrive the best model